In [ ]:
!pip install easyocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 26.1 MB/s eta 0:00:00


In [ ]:
!pip install opencv-python

In [ ]:
!pip install streamlit

In [ ]:
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.8/731.8 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-7.0.1-py3-none-any.whl size=21122 sha256=9f4c7f33fb3dad5e46a108d72d869ead2b3d79324af737dcaf6b92c9f4ec5707
  Stored in directory: /root/.cache/pip/wheels/3b/32/0e/27789b6fde02bf2b320d6f1a0fd9e1354b257c5f75eefc29bc
Successfully built pyngrok


In [ ]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 2.701s

3 packages are looking for funding
  run `npm fund` for details

found 1 moderate severity vulnerability
  run `npm audit fix` to fix them, or `npm audit` for details


In [ ]:
!pip install streamlit-option-menu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 799.2/799.2 kB 10.4 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
import streamlit as st
from streamlit_option_menu import option_menu
import pandas as pd
import re
import os
import numpy as np
import easyocr
from easyocr import Reader
from PIL import Image,ImageDraw
import cv2
import matplotlib.pyplot as plt
import sqlite3

#SETTING PAGE CONFIGURATION
icon = Image.open("business_card.png")
st.set_page_config(
    page_title="Bizcardx: Extracting Bussiness card Data with EasyOCR",
    page_icon= icon,
    layout="wide",
    initial_sidebar_state="expanded")
st.markdown("<h1 style='text-align:center;color:black;'>Bizcardx: Extracting Bussiness card Data with EasyOCR</h1>",unsafe_allow_html=True)

#CREATING OPTION MENU
with st.sidebar:
    selected =option_menu(
        menu_title = None,
        options=["HOME","UPLOAD & EXTRACT","MODIFY & DELETE"],
        icons= ["house","cloud-upload-fill","pencil-square"],
        menu_icon ="cast",
        default_index= 0,
        orientation="horizontal",
    )
#INITIALIZING THE EASYOCR READER
reader = easyocr.Reader(['en'],gpu = False)          #verbose=False
# CONNECTING WITH MYSQL DATABASE
import sqlite3
con = sqlite3.connect("Bizcardx.db")
mycursor = con.cursor()
# TABLE CREATION
mycursor.execute('''CREATE TABLE if not exists bizcard
                 (
                 Company_Name TEXT,
                 Card_Holder TEXT,
                 Designation TEXT,
                 Mobile_Number VARCHAR(50),
                 email TEXT,
                 website TEXT,
                 Area TEXT,
                 City TEXT,
                 State TEXT,
                 Pincode VARCHAR(10),
                 image LONGBLOB)''')


#HOME MENU
if selected == "HOME":
    col1,col2 = st.columns(2)
    with col1:
        st.markdown("[**TECHNOLOGIES_USED:**] PYTHON,EASYOCR,STREAMLIT,SQL & pandas")
        st.markdown("[**OVERVIEW:**]In this streamlit web app we can upload the business card image and extract the data in the image")
    st.header('''OCR stands for "Optical Character Recognition."
                        It is a technology that recognizes text within a digital image.
                 It is commonly used to recognize text in scanned documents and images.
                 OCR software can be used to convert a physical paper document,
                or an image into an accessible electronic version with text.''')
    st.subheader('''Importance of OCR:
                OCR technology solves the problem by converting text images into text data that can be analyzed by other business software.
                You can then use the data to conduct analytics, streamline operations, automate processes, and improve productivity.  ''')
    with col2:
      st.image("business_card.png")

# UPLOAD & EXTRACTING DATA FROM IMAGE
if selected == "UPLOAD & EXTRACT":
    st.title("uploading image")
    st.markdown("---")
    uploaded_card =st.file_uploader("please upload the Image",label_visibility= "collapsed",type=["png","jpg","jpeg"])
    if uploaded_card is not None:
      def save_card(uploaded_card):
        with open(os.path.join(uploaded_card.name),"wb") as f:
          f.write(uploaded_card.getbuffer())
      save_card(uploaded_card)
      def image_preview(img,bounds):
            #unpack the bounding box
            for (bbox,text,prob) in bounds:
                (t1,tr,br,b1) = bbox
                t1 = (int(t1[0]),int(t1[1]))
                tr = (int(tr[0]),int(t1[1]))
                br = (int(br[0]),int(br[1]))
                b1 = (int(b1[0]),int(b1[1]))
                cv2.rectangle(img,t1,br,(0,255,0),2)
                cv2.putText(img,text,(t1[0],t1[1] - 10),
                cv2.FONT_HERSHEY_COMPLEX, 0.7, (255,0,0),2)
            plt.rcParams['figure.figsize'] = (15,15)
            plt.axis('off')
            plt.imshow(img)
# DISPLAYING THE UPLOADED CARD
      col1,col2 = st.columns(2, gap = "large")
      with col1:
        st.markdown("### You have uploaded the card")
        st.image(uploaded_card)

#DISPLAYING THE CARD WITH HIGHLIGHTS
      with col2:
        with st.spinner("please wait processing image..."):

          saved_image = os.path.join(os.getcwd()+"/"+ uploaded_card.name)
          img = cv2.imread(saved_image)
          pil_image = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
          img_array = cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)
          bounds = reader.readtext(img_array)


#Displaying image with bounding boxes
        def draw_boxes(img, bounds, color='yellow', width=2):
            img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))  # Convert NumPy array to PIL Image
            draw = ImageDraw.Draw(img)
            for bound in bounds:
              p0, p1, p2, p3 = bound[0]
              draw.line([*p0, *p1, *p2, *p3, *p0], fill=color, width=width)
            return img
        result = draw_boxes(img, bounds)
        st.image(result)

       # CONVERTING IMAGE TO BINARY
        def img_to_binary(file):
                with open(file, "rb") as file:
                    BinaryData = file.read()
                return BinaryData

        # FUNCTION TO GET DATA
        def extract(l):
          data = {"Company_Name": [],"Card_Holder": [],"Designation": [],"Mobile_Number": [],"email": [],"website": [],"Area": [],"City": [],"State": [],"Pincode": []}

          data['Card_Holder'].append(l[0])
          l.pop(0)
          data['Designation'].append(l[0])
          l.pop(0)
          for j in range(3):
              for i in l:

                  if '+' in i or '-' in i:
                      data["Mobile_Number"].append(l.pop(l.index(i)))
                  elif i.isnumeric():
                      data['Pincode'].append(l.pop(l.index(i)))
                  elif '@' in i:
                      data['email'].append(l.pop(l.index(i)))
                  elif ('www' in i.lower() or '.com' in i.lower()) and '@' not in i.lower():
                      data['website'].append(l.pop(l.index(i)))
                  elif ',' in i or ';' in i:
                      k = i.replace(';',',').replace(',,',',')
                      k=k.split(',')
                      if len(k) > 2:
                          data['Area'].append(k[0])
                          data['City'].append(k[1])
                          data['State'].append(k[2])
                          l.pop(l.index(i))
                  elif len(i.split())==2:
                      if i.split()[1].isnumeric():
                          data['State'].clear()
                          data['Pincode'].clear()
                          data['State'].append(i.split()[0])
                          data['Pincode'].append(i.split()[1])
                          l.pop(l.index(i))
          if len(l) <= 3:
              data['Company_Name'].append(' '.join(l))
          else:
              data['Company_Name'].append(l.pop(2)+l.pop(2))
              data["Area"].append(l.pop(2)+l.pop(0))
              data["City"].append(l.pop(0))
          return {i:' '.join(data[i]) for i in data}
        data =[extract([i[1] for i in bounds])]
        #data = st.write([i[1] for i in bounds])
        st.write(data)
        # FUNCTION TO CREATE DATAFRAME
        df = pd.DataFrame(data)
        df ['image']= img_to_binary(saved_image)
        st.dataframe(df)
        st.success("### DATA EXTRACTED SUCCESSFULLY", icon="✅")

        if st.button("Upload to Database"):
          for i, row in df.iterrows():
              sql = """INSERT INTO bizcard('Company_Name', 'Card_Holder', 'Designation', 'Mobile_Number', 'email', 'website',
                                        'Area', 'City', 'State', 'Pincode', 'image') VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"""
              mycursor.execute(sql, tuple(row))
              con.commit()
          st.success("### Uploaded to Database successfully",icon="✅")
          #checking whether the data inserted into sql database
        #S = mycursor.execute('SELECT* FROM bizcard')
        #d = S.fetchall()
        #st.write(d)
# MODIFY AND
if selected == "MODIFY & DELETE":
  col1,col2,col3 = st.columns([3,3,2])
  col2.markdown("## Alter or Delete the data here")
  column1,column2 = st.columns(2,gap="large")
  try:
      with column1:
        mycursor.execute("SELECT Card_Holder FROM bizcard")
        result = mycursor.fetchall()
        business_cards = {}
        for row in result:
          business_cards[row[0]]= row[0]
        selected_card = st.selectbox("select a Card_Holder name to update",list(business_cards.keys()))
        st.markdown("#### Update or modify the data given below")
        mycursor.execute("SELECT Company_Name,Card_Holder,Designation,Mobile_Number,email,website,Area,City,State,Pincode FROM bizcard WHERE Card_Holder=?",(selected_card,))
        result = mycursor.fetchone()
        # Displaying the information
        Company_Name = st.text_input("Company_Name",result[0])
        Card_Holder = st.text_input("Card_Holder",result[1])
        Designation = st.text_input("Designation",result[2])
        Mobile_Number = st.text_input("Mobile_Number",result[3])
        email = st.text_input("email",result[4])
        website = st.text_input("website",result[5])
        Area = st.text_input("Area",result[6])
        City = st.text_input("City",result[7])
        State = st.text_input("State",result[8])
        Pincode = st.text_input("pincode",result[9])
        if st.button("Commit changes to DB"):
          # update the information for the selected business_card in the database
          mycursor.execute("""UPDATE bizcard SET Company_Name = ?,Card_Holder = ?,Designation = ?,Mobile_Number = ?,email =?,website =?,Area = ?,City = ?,State = ?,Pincode = ?
                          WHERE Card_Holder = ?""",(Company_Name,Card_Holder,Designation,Mobile_Number,email,website,Area,City,State,Pincode,selected_card))
          con.commit()
          st.success("Information updated in database Succcessfully.")
      with column2:
        mycursor.execute("SELECT Card_Holder FROM bizcard")
        result = mycursor.fetchall()
        business_cards = {}
        for row in result:
          business_cards[row[0]] = row[0]
        selected_card = st.selectbox("select a Card_Holder name to Delete",list(business_cards.keys()))
        st.write(f"### You have selected: green[**{selected}'s**] card to delete")
        st.write("Proceed to delete this card")
        if st.button("Delete the business_card"):
          mycursor.execute(f"DELETE FROM bizcard WHERE Card_Holder = '{selected_card}'")
          con.commit()
          st.success("Business card information deleted from database.")
  except:
    st.warning("There is no data available in the database")












Overwriting app.py


In [ ]:
!streamlit run /content/app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

35.185.27.206
npx: installed 22 in 2.908s
your url is: https://dull-corners-chew.loca.lt
